In [1]:
import sympy as sp
from sympy.matrices import Matrix
#import numpy as np

# pretty print numpy matrices
#np.set_printoptions(formatter={'float': '{: 0.5f}'.format})

In [2]:
th1, th2, th3, th4, th5, th6 = sp.symbols('th1:7')

# the DH paramters for KUKA robot as per written document
# alpha, a, d, theta (adjustmet)
# the angles we will use to control the robot will be added to the theta paramters
DH = [[0       , 0     , 0.75 , th1          ],
      [-sp.pi/2, 0.35  , 0    , th2 - sp.pi/2],
      [0       , 1.25  , 0    , th3          ],
      [-sp.pi/2, -0.054, 1.5  , th4          ],
      [sp.pi/2 , 0     , 0    , th5          ],
      [-sp.pi/2, 0     , 0    , th6          ],
      [0       , 0     , 0.303, 0            ]]

In [3]:
# a help function that builds a transformation matrix given the 4 DH paramters for that joint
def LinkTransform(params):
    # params is a list of length 4: alpha, a, d, theta in this order
    alpha = params[0]
    a = params[1]
    d = params[2]
    theta = params[3]
    # returns a Matrix of transformation for the given DH paramters
    return Matrix([[sp.cos(theta)              , -sp.sin(theta)             , 0             , a],
                   [sp.sin(theta)*sp.cos(alpha), sp.cos(theta)*sp.cos(alpha), -sp.sin(alpha), -sp.sin(alpha)*d],
                   [sp.sin(theta)*sp.sin(alpha), sp.cos(theta)*sp.sin(alpha), sp.cos(alpha) , sp.cos(alpha)*d],
                   [0                          , 0                          , 0             , 1]])

In [4]:
T34 = LinkTransform(DH[3])
T45 = LinkTransform(DH[4])
T56 = LinkTransform(DH[5])
T36 = sp.simplify(T34 * T45 * T56)
print(T36[0:3,0:3])

Matrix([
[-sin(th4)*sin(th6) + cos(th4)*cos(th5)*cos(th6), -sin(th4)*cos(th6) - sin(th6)*cos(th4)*cos(th5), -sin(th5)*cos(th4)],
[                              sin(th5)*cos(th6),                              -sin(th5)*sin(th6),           cos(th5)],
[-sin(th4)*cos(th5)*cos(th6) - sin(th6)*cos(th4),  sin(th4)*sin(th6)*cos(th5) - cos(th4)*cos(th6),  sin(th4)*sin(th5)]])


In [14]:
# builds a transformation matrix for the orientation adjustment of the gripper
# so that we are consistent with the URDF representaiton of the gripper
def GripperAdjust(r_z = sp.pi, r_y = -sp.pi/2):
    R_z = Matrix([[sp.cos(r_z), -sp.sin(r_z), 0, 0],
                  [sp.sin(r_z), sp.cos(r_z) , 0, 0],
                  [0          , 0           , 1, 0],
                  [0          , 0           , 0, 1]])
    R_y = Matrix([[sp.cos(r_y) , 0          , sp.sin(r_y), 0],
                  [0           , 1          , 0          , 0],
                  [-sp.sin(r_y), 0          , sp.cos(r_y), 0],
                  [0           , 0          , 0          , 1]])
    return sp.simplify(R_z * R_y)

In [21]:
T01 = LinkTransform(DH[0])
print('T01 = '+str(sp.simplify(T01))+'\n')
T12 = LinkTransform(DH[1])
print('T12 = '+str(sp.simplify(T12))+'\n')
T23 = LinkTransform(DH[2])
print('T23 = '+str(sp.simplify(T23))+'\n')
T34 = LinkTransform(DH[3])
print('T34 = '+str(sp.simplify(T34))+'\n')
T45 = LinkTransform(DH[4])
print('T45 = '+str(sp.simplify(T45))+'\n')
T56 = LinkTransform(DH[5])
print('T56 = '+str(sp.simplify(T56))+'\n')
T6G = LinkTransform(DH[6])
print('T6G = '+str(sp.simplify(T6G))+'\n')
print('TGA = '+str(GripperAdjust()))

T01 = Matrix([
[cos(th1), -sin(th1), 0,    0],
[sin(th1),  cos(th1), 0,    0],
[       0,         0, 1, 0.75],
[       0,         0, 0,    1]])

T12 = Matrix([
[sin(th2),  cos(th2), 0, 0.35],
[       0,         0, 1,    0],
[cos(th2), -sin(th2), 0,    0],
[       0,         0, 0,    1]])

T23 = Matrix([
[cos(th3), -sin(th3), 0, 1.25],
[sin(th3),  cos(th3), 0,    0],
[       0,         0, 1,    0],
[       0,         0, 0,    1]])

T34 = Matrix([
[ cos(th4), -sin(th4), 0, -0.054],
[        0,         0, 1,    1.5],
[-sin(th4), -cos(th4), 0,      0],
[        0,         0, 0,      1]])

T45 = Matrix([
[cos(th5), -sin(th5),  0, 0],
[       0,         0, -1, 0],
[sin(th5),  cos(th5),  0, 0],
[       0,         0,  0, 1]])

T56 = Matrix([
[ cos(th6), -sin(th6), 0, 0],
[        0,         0, 1, 0],
[-sin(th6), -cos(th6), 0, 0],
[        0,         0, 0, 1]])

T6G = Matrix([
[1, 0, 0,     0],
[0, 1, 0,     0],
[0, 0, 1, 0.303],
[0, 0, 0,     1]])

TGA = Matrix([
[0,  0, 1, 0],
[0, -1, 0, 0],


In [24]:
T0G = sp.simplify(T01*T12*T23*T34*T45*T56*T6G)
print('T0G unajusted = '+str(T0G))

T0G unajusted = Matrix([
[((sin(th1)*sin(th4) + sin(th2 + th3)*cos(th1)*cos(th4))*cos(th5) + sin(th5)*cos(th1)*cos(th2 + th3))*cos(th6) - (-sin(th1)*cos(th4) + sin(th4)*sin(th2 + th3)*cos(th1))*sin(th6), -((sin(th1)*sin(th4) + sin(th2 + th3)*cos(th1)*cos(th4))*cos(th5) + sin(th5)*cos(th1)*cos(th2 + th3))*sin(th6) + (sin(th1)*cos(th4) - sin(th4)*sin(th2 + th3)*cos(th1))*cos(th6), -(sin(th1)*sin(th4) + sin(th2 + th3)*cos(th1)*cos(th4))*sin(th5) + cos(th1)*cos(th5)*cos(th2 + th3), -0.303*sin(th1)*sin(th4)*sin(th5) + 1.25*sin(th2)*cos(th1) - 0.303*sin(th5)*sin(th2 + th3)*cos(th1)*cos(th4) - 0.054*sin(th2 + th3)*cos(th1) + 0.303*cos(th1)*cos(th5)*cos(th2 + th3) + 1.5*cos(th1)*cos(th2 + th3) + 0.35*cos(th1)],
[ ((sin(th1)*sin(th2 + th3)*cos(th4) - sin(th4)*cos(th1))*cos(th5) + sin(th1)*sin(th5)*cos(th2 + th3))*cos(th6) - (sin(th1)*sin(th4)*sin(th2 + th3) + cos(th1)*cos(th4))*sin(th6), -((sin(th1)*sin(th2 + th3)*cos(th4) - sin(th4)*cos(th1))*cos(th5) + sin(th1)*sin(th5)*cos(th2 + th3))*sin(th

In [25]:
T0GA = sp.simplify(T0G*GripperAdjust())
print('T0G ajusted = '+str(T0GA))

T0G ajusted = Matrix([
[-(sin(th1)*sin(th4) + sin(th2 + th3)*cos(th1)*cos(th4))*sin(th5) + cos(th1)*cos(th5)*cos(th2 + th3), ((sin(th1)*sin(th4) + sin(th2 + th3)*cos(th1)*cos(th4))*cos(th5) + sin(th5)*cos(th1)*cos(th2 + th3))*sin(th6) - (sin(th1)*cos(th4) - sin(th4)*sin(th2 + th3)*cos(th1))*cos(th6), ((sin(th1)*sin(th4) + sin(th2 + th3)*cos(th1)*cos(th4))*cos(th5) + sin(th5)*cos(th1)*cos(th2 + th3))*cos(th6) + (sin(th1)*cos(th4) - sin(th4)*sin(th2 + th3)*cos(th1))*sin(th6), -0.303*sin(th1)*sin(th4)*sin(th5) + 1.25*sin(th2)*cos(th1) - 0.303*sin(th5)*sin(th2 + th3)*cos(th1)*cos(th4) - 0.054*sin(th2 + th3)*cos(th1) + 0.303*cos(th1)*cos(th5)*cos(th2 + th3) + 1.5*cos(th1)*cos(th2 + th3) + 0.35*cos(th1)],
[-(sin(th1)*sin(th2 + th3)*cos(th4) - sin(th4)*cos(th1))*sin(th5) + sin(th1)*cos(th5)*cos(th2 + th3), ((sin(th1)*sin(th2 + th3)*cos(th4) - sin(th4)*cos(th1))*cos(th5) + sin(th1)*sin(th5)*cos(th2 + th3))*sin(th6) + (sin(th1)*sin(th4)*sin(th2 + th3) + cos(th1)*cos(th4))*cos(th6), ((sin(th1)*s

In [ ]:
# builds the transformation matrices based on the DH and the theta paramters
# it resurns the final transformation matrix
def ChainLinkTransform(DH, show = False):
    for i in range(len(DH)):
        T = LinkTransform(DH[i])
        if i == 0:
            result = T
        else:
            result = sp.simplify(result * T)

        if show:
            print("T%d_%d = " % (i, i+1))
            print(T)
            print("T0_%d = " % (i+1))
            print(result)
            
    return sp.simplify(result * GripperAdust())

In [ ]:
# extracts the position and orientation from the quaternion matrix
def OrientationFromQuaternion(Q):
    pos = Q[:,3].T
    orient = [sp.atan2(Q[2,1], Q[2,2]),
              sp.atan2(-Q[2,0], sp.sqrt(Q[0,0]**2 + Q[1,0]**2)),
              sp.atan2(Q[1,0], Q[0,0])]
    
    return pos[0:3], orient

In [ ]:
def CalculateEffector(sym, thetas):
    res = sym.evalf(subs={th1: thetas[0], th2: thetas[1], th3: thetas[2], 
                          th4: thetas[3], th5: thetas[4], th6: thetas[5]})
    pos, orient = OrientationFromQuaternion(res)
    print("pos    = "+str(pos))
    print("orient = "+str(orient))  

In [ ]:
symtransf = ChainLinkTransform(DH)

In [ ]:
th = [0, 0, 0, 0, 0, 0]
CalculateEffector(symtransf, th)

ROS Reported:<br>
`Trans = [2.153, 0.000, 1.947]
Rot = [0.000, 0.000, 0.000]`

In [ ]:
th = [0.99, 0, 0, 0, 0, 0, 0]
CalculateEffector(symtransf, th)

ROS Reported:<br>
Trans = [1.173, 1.805, 1.947] <br>
Rot = [0.000, 0.000, 0.994]

In [ ]:
th = [0.99, 0, 0, 0, 0, 0.72, 0]
CalculateEffector(DH, th)

ROS Reported:<br>
Trans = [1.173, 1.805, 1.947] <br>
Rot = [0.720, 0.0, 0.994]

In [ ]:
th = [0.99, 0, 0, 0, 0.49, 0.72, 0]
CalculateEffector(DH, th)

ROS Reported:<br>
Trans = [1.154, 1.775, 1.804] <br>
Rot   = [0.720, 0.489, 0.994]